In [0]:
%pip install databricks-vectorsearch

In [0]:
%sql
create table benmackenzie_catalog.mlflow3.parsed_gisa_document as
SELECT
  path,
  parsed_json:document:pages AS pages,
  parsed_json:document:elements AS elements,
  parsed_json:document:_corrupted_data AS _corrupted_data
FROM (
  SELECT
    path,
    ai_parse_document(content) AS parsed_json
  FROM READ_FILES('/Volumes/benmackenzie_catalog/mlflow3/documents', format => 'binaryFile')
)

In [0]:
%sql
CREATE OR REPLACE TABLE benmackenzie_catalog.mlflow3.gisa_document_pages as
SELECT pos as page_number, string(value:content) as content
FROM benmackenzie_catalog.mlflow3.parsed_gisa_document AS t,
LATERAL variant_explode(t.pages) AS v;

In [0]:
%sql
select * from benmackenzie_catalog.mlflow3.gisa_document_pages

In [0]:
%sql

ALTER TABLE benmackenzie_catalog.mlflow3.gisa_document_pages
SET TBLPROPERTIES (delta.enableChangeDataFeed = true)


##Run following cells or use UI

In [0]:
from databricks.vector_search.client import VectorSearchClient

# Get the vector search index
vsc = VectorSearchClient(disable_notice=True)

In [0]:
vsc.create_delta_sync_index_and_wait(
        endpoint_name="one-env-shared-endpoint-12",
        index_name="benmackenzie_catalog.mlflow3.gisa_document_pages_index",
        primary_key="page_number",
        source_table_name="benmackenzie_catalog.mlflow3.gisa_document_pages",
        pipeline_type="triggered",
        embedding_source_column="content",
        embedding_model_endpoint_name="databricks-gte-large-en"
    )

In [0]:
index = vsc.get_index(endpoint_name="one-env-shared-endpoint-12", index_name="benmackenzie_catalog.mlflow3.gisa_document_pages_index")
index.similarity_search(columns=["content", "page_number"], query_text="What are the applicable jurisdiction for Enhanced Accident Benefits coverage?")
